When we need read video into frame tensors, we often use torchvision.io.read_video. However, I found it is very very slow and cpu intensive. After reading https://pytorch.org/vision/stable/io.html#torchvision.io.VideoReader , it says pytorch has a a high-performance lower-level API (torchvision.io.VideoReader) for more fine-grained control compared to the read_video function.

I made a quick benchmark and found that VideoReader is sometimes much faster (about 100 faster) and less cpu intensive.

For how to use VideoReader, see https://pytorch.org/vision/stable/generated/torchvision.io.VideoReader.html#torchvision.io.VideoReader

In [1]:
import torchvision
import time
import torch

video_path = "train_data/1.mp4"

ts = time.time()
reader = torchvision.io.VideoReader(video_path, "video")
frames = []
for frame in reader:
    frames.append(frame)
te = time.time()
print(f"torchvision.io.VideoReader run time: {te - ts}")

ts = time.time()
video, audio, info = torchvision.io.read_video(
    video_path,
    start_pts=0.0,
    end_pts=None,
    pts_unit="sec",
    output_format="TCHW",
)
te = time.time()
print(f"io.read_video run time: {te - ts}")
print(info)

assert(len(frames) == len(video))

print([torch.equal(frames[i]['data'], video[i]) for i in range(len(video))])



torchvision.io.VideoReader run time: 0.4836447238922119
io.read_video run time: 48.35976314544678
{'video_fps': 30.0, 'audio_fps': 44100}
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
